In [1]:
#### 12/06勉強会でのコード
import numpy as np

In [2]:
# グリッド設定
m = 3  # 内生的状態変数aのグリッド数 (a1, a2, a3)
n = 2  # 外生的状態変数yのグリッド数 (y_H, y_L)
epsilon = 1e-5  # 収束基準


In [3]:
# 外生的状態 y の遷移確率行列 (例: 一様分布)
p = np.array([[0.9, 0.1],
              [0.1, 0.9]])  # y_H と y_L 間の遷移確率

In [4]:
a_grid = np.array([12, 24, 36])  # [a1, a2, a3]
y_grid = np.array([1, 0.5])     # [Y_high, Y_low]

In [5]:
# 政策関数のグリッド
pf = np.array([[20, -48],    # g(a1, y_H), g(a1, y_L)
               [32, 15],    # g(a2, y_H), g(a2, y_L)
               [144, 27]])  # g(a3, y_H), g(a3, y_L)

In [6]:
# 初期分布 f^{(0)}_{i,j}
f = np.full((m, n), 1 / (m * n))
f = f.T
f= f.reshape(6, 1)

In [7]:
f_new = np.zeros((m, n))
#for j in range(n):  #次期のyのindexについて繰り返し
pf_t = pf.T
target_a = pf_t.reshape(6, 1)  # 政策関数による次期a

In [8]:
# 定常分布の計算ループ
max_diff = epsilon + 1  # 初期化: epsilon より大きくしてループを開始
iteration = 0

while max_diff >= epsilon:
    f_new = np.zeros((m, n))
    # pf_t = pf.T
    # target_a = pf_t.reshape(6, 1)  # 政策関数による次期a    

    for j in range(n):  #次期のyのindexについて繰り返し
        for i in range(m): #次期のaのindexについて繰り返し

            if i == 0:
                target_match = np.zeros((6,1))
                weight = np.zeros((6,1))
                ind_r = np.where((target_a > a_grid[i]) & (target_a <= a_grid[i+1]))[0]
                target_match[ind_r] = 1
                ind_l = np.where(target_a <= a_grid[i])[0]
                target_match[ind_l] = 2

                weight[ind_r] = (a_grid[i+1] - target_a[ind_r]) / (a_grid[i+1] - a_grid[i])
                weight[ind_l] = 1

            elif i == len(a_grid)-1:
                target_match = np.zeros((6,1))
                weight = np.zeros((6,1))
                ind_r = np.where(target_a > a_grid[i])[0]
                target_match[ind_r] = 1
                ind_l = np.where((target_a > a_grid[i-1]) & (target_a <= a_grid[i]))[0]
                target_match[ind_l] = 2

                weight[ind_r] = 1
                weight[ind_l] = (target_a[ind_l]    - a_grid[i-1]  ) / (a_grid[i] - a_grid[i-1])

            else:
                target_match = np.zeros((6,1))
                weight = np.zeros((6,1))
                ind_r = np.where((target_a > a_grid[i]) & (target_a <= a_grid[i+1]))[0]
                target_match[ind_r] = 1
                ind_l = np.where((target_a > a_grid[i-1]) & (target_a <= a_grid[i]))[0]
                target_match[ind_l] = 2

                weight[ind_r] = (a_grid[i+1] - target_a[ind_r]) / (a_grid[i+1] - a_grid[i])
                weight[ind_l] = (target_a[ind_l]    - a_grid[i-1]  ) / (a_grid[i] - a_grid[i-1])


            a = p[0, j]
            b = p[1, j]


            # ベクトルを生成
            vector = np.concatenate([np.repeat(a, m), np.repeat(b, m)])
            vector = vector.reshape(6, 1)


            f_new[i,j] = np.sum(vector * f * weight)
            # f_new = f_new.T
            # f_new= f_new.reshape(6, 1)

    # 収束の確認
    f_new = f_new.T
    f_new= f_new.reshape(6, 1)    
    max_diff = np.max(np.abs(f_new - f))
    print(f"Iteration {iteration + 1}: max difference = {max_diff}")

    # 更新
    f = f_new
    iteration += 1

# 最終イタレーション回数の出力
print(f"Total iterations until convergence: {iteration}")

# 定常分布の出力
print("Stationary distribution f_{i,j}:")
print(f)

Iteration 1: max difference = 0.10138888888888889
Iteration 2: max difference = 0.08833333333333332
Iteration 3: max difference = 0.032179687500000054
Iteration 4: max difference = 0.010823227719907416
Iteration 5: max difference = 0.0032067686631944564
Iteration 6: max difference = 0.0008705006857036152
Iteration 7: max difference = 0.0002646113187436683
Iteration 8: max difference = 0.00010705515376152608
Iteration 9: max difference = 4.4198063778089214e-05
Iteration 10: max difference = 1.892597364522297e-05
Iteration 11: max difference = 8.851825837374871e-06
Total iterations until convergence: 11
Stationary distribution f_{i,j}:
[[0.06176441]
 [0.06068467]
 [0.37755091]
 [0.39116845]
 [0.05489627]
 [0.05393528]]
